In [1]:
from dotenv import load_dotenv
load_dotenv("OPENAI_API_KEY")
load_dotenv("OPENAI_ORG_ID")
import os
import openai
import json

In [2]:
openai.organization = os.getenv("OPENAI_ORG_ID")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
def stream_response(response):
    for chunk in response:
        if chunk["choices"][0] == "stop":
            break
        if "content" not in chunk["choices"][0]["delta"]:
            continue
        print(chunk["choices"][0]["delta"]["content"], end="")

In [4]:
def get_current_weather(location, is_clothing_index, unit="摂氏"):
    """受け取った場所の天気を返す"""
    if is_clothing_index:
        weather_info = {
            "location": location,
            "temperature": "-100",
            "unit": unit,
            "forecast": ["大雪", "強風"],
            "is_clothing_index": "熊の毛皮を最低10枚は重ね着してください"
        }
    else:
        weather_info = {
            "location": location,
            "temperature": "27",
            "unit": unit,
            "forecast": ["大雨", "落雷"],
        }
    return json.dumps(weather_info)

In [5]:
def run_conversation(user_message: str):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": user_message}],
        functions=[
            {
                "name": "get_current_weather",
                "description": "与えられた場所から現在の天気を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "都道府県と市町村。例えば、東京都。",
                        },
                        "unit": {
                            "type": "string",
                            "description": "温度の単位。基本は摂氏。",
                            "enum": ["摂氏", "華氏"]
                        },
                        "is_clothing_index": {
                            "type": "boolean",
                            "description": "どんな服装が良いかを含めるかどうか。例えば、True",
                            "enum": [True, False]
                        }
                    },
                    "required": ["location", "is_clothing_index"],
                },
            }
        ],
        function_call="auto",
    )

    message = response["choices"][0]["message"]

    if message.get("function_call"):
        function_name = message["function_call"]["name"]
        arguments = json.loads(message["function_call"]["arguments"])

        function_response = get_current_weather(
            location=arguments.get("location"),
            unit=arguments.get("unit"),
            is_clothing_index=arguments.get("is_clothing_index"),
        )

        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": user_message},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
            stream=True,
        )
        return second_response

In [8]:
response = run_conversation("現在の渋谷区の天気を教えて！")
stream_response(response)

現在の渋谷区の天気は、27℃で、大雨と落雷が予想されています。

In [9]:
response = run_conversation("現在の渋谷区の天気を教えて、服装指数についても教えてください")
stream_response(response)

現在の渋谷区の天気は、大雪で強風です。気温は-100℃です。服装指数は、熊の毛皮を最低10枚は重ね着してくださいとなっています。十分に暖かな服装で外出するようにしてください。